# Parameter Sensitivity Analysis

This notebook explores how ERH behavior changes with different parameters:
- Judgment system parameters (bias_strength, noise_scale)
- Action space parameters (complexity_dist, moral_ambiguity_factor)
- Ethical prime selection criteria (importance_quantile, strategy)

We identify which parameters most strongly affect whether ERH is satisfied.


In [ ]:
import sys
sys.path.append('..')
import numpy as np
import matplotlib.pyplot as plt

from core.action_space import generate_world
from core.judgement_system import BiasedJudge, evaluate_judgement
from core.ethical_primes import select_ethical_primes, compute_Pi_and_error, analyze_error_growth
from visualization.plots import setup_paper_style

setup_paper_style()

# Sweep bias_strength
bias_strengths = np.linspace(0, 0.5, 11)
exponents = []

for bias in bias_strengths:
    actions = generate_world(num_actions=1000, random_seed=42)
    judge = BiasedJudge(bias_strength=bias, noise_scale=0.1)
    evaluate_judgement(actions, judge, tau=0.3)
    primes = select_ethical_primes(actions)
    
    if len(primes) > 10:
        Pi_x, B_x, E_x, x_vals = compute_Pi_and_error(primes, X_max=100)
        analysis = analyze_error_growth(E_x, x_vals)
        exponents.append(analysis['estimated_exponent'])
    else:
        exponents.append(np.nan)

# Plot
plt.figure(figsize=(10, 6))
plt.plot(bias_strengths, exponents, 'o-', linewidth=2)
plt.axhline(y=0.5, color='r', linestyle='--', label='ERH threshold (α=0.5)')
plt.xlabel('Bias Strength')
plt.ylabel('Estimated Exponent α')
plt.title('Sensitivity to Bias Strength')
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig('../output/figures/04_sensitivity_bias.pdf', dpi=300)
plt.show()

print("Parameter sensitivity analysis complete!")
